In [ ]:
import csv
from matplotlib import pyplot as plt
from matplotlib import cm
import numpy as np

# Colormaps: paired
# Extract colors from colormap, to use at will
colormap = cm.get_cmap('Paired')
colors = [colormap(i) for i in np.linspace(0, 1, 12)]


PNO_time = dict([("1%", []), ("2%", []), ("5%", []), ("5%_noH2O", [])])
PNO_yield = dict([("1%", []), ("2%", []), ("5%", []), ("5%_noH2O", [])])

with open('../data/PNO_Optimisation.csv', 'r') as data_file:
    plots = csv.reader(data_file, delimiter=';')
    next(plots)
    next(plots)
    next(plots)
    for row in plots:
        PNO_time["5%_noH2O"].append(row[0])
        PNO_yield["5%_noH2O"].append(row[1])
        PNO_time["1%"].append(row[2])
        PNO_yield["1%"].append(row[3])
        PNO_time["2%"].append(row[4])
        PNO_yield["2%"].append(row[5])
        PNO_time["5%"].append(row[6])
        PNO_yield["5%"].append(row[7])

# Cast str to float
PNO_yield = {j:[float(i) for i in PNO_yield[j] if i] for j in PNO_yield}
PNO_time = {j:[float(i) for i in PNO_time[j] if i] for j in PNO_time}

In [ ]:
# Setup axes
axis = plt.gca()
axis.set_axisbelow(True)
axis.set_xlabel("Time (min)")
axis.set_xlim(0, 60)
axis.set_ylim(-5, 100)
axis.set_ylabel("Yield (%)")

# Plot lines

plt.plot(PNO_time['1%'], PNO_yield['1%'],'o', zorder=2.6, color=colors[5])
plt.plot(PNO_time['2%'], PNO_yield['2%'],'o', zorder=2.6, color=colors[1])
plt.plot(PNO_time['5%'], PNO_yield['5%'],'o', zorder=2.6, color=colors[3])
plt.plot(PNO_time['5%_noH2O'], PNO_yield['5%_noH2O'],'o', zorder=2.6, color=colors[7])

# Save file
plt.savefig("../data/PNO_Optimization.png", dpi=300)